In [ ]:
import pandas as pd
import requests
from keras.models import Model, load_model

# Goal
Predict the daily number of cases for the given countries, regions and intervention plan.

This model is described in [From Prediction to Prescription: Evolutionary Optimization of Non-Pharmaceutical Interventions in the COVID-19 Pandemic](https://arxiv.org/abs/2005.13766) in Section 5 - Data-Driven Predictive Model.

Predictions from this model trained on the latest data are visible here: [How AI Makes Intervention Recommendations](https://evolution.ml/demos/npidashboard/). For more information see [Augmenting Human Decision Making
Optimizing COVID-19 Interventions](https://evolution.ml/esp/npi/).

## Intervention plan
An intervention plan consists in a list of containment and closure policies, as well as health system policies. See https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/codebook.md for more details. The following policies are considered:

In [ ]:
NPI_COLUMNS = ['C1_School closing',
               'C2_Workplace closing',
               'C3_Cancel public events',
               'C4_Restrictions on gatherings',
               'C5_Close public transport',
               'C6_Stay at home requirements',
               'C7_Restrictions on internal movement',
               'C8_International travel controls',
               'H1_Public information campaigns',
               'H2_Testing policy',
               'H3_Contact tracing']

## Example
For the given input, provide a model that can produce the expected output

### Given input
One row represents a country, region and day. For each row, the interventions that were in place on this day for this region and country are provided. Interventions can change from one day to the other for each country and region.

In [ ]:
EXAMPLE_INPUT_FILE = "../../validation/data/2020-08-01_2020-08-04_ip.csv"

In [ ]:
prediction_input_df = pd.read_csv(EXAMPLE_INPUT_FILE,
                                  parse_dates=['Date'],
                                  encoding="ISO-8859-1")

In [ ]:
prediction_input_df.head()

### Expected output
For each row that was provided in input, i.e. for each country, region and day, the output should contain an additional `PredictedDailyNewCases` column with the predicted number of cases for that day, region and country. It is possible to leave `PredictedDailyNewCases` empty or NaN, or to remove the row, in case no predition is available.

In [ ]:
EXAMPLE_OUTPUT_FILE = "../../2020-08-01_2020-08-04_predictions_example.csv"

In [ ]:
prediction_output_df = pd.read_csv(EXAMPLE_OUTPUT_FILE,
                                   parse_dates=['Date'],
                                   encoding="ISO-8859-1")

In [ ]:
prediction_output_df.head()

### Evaluation
Predictions will be evaluated on a period of 4 weeks **after** submision against the actual daily change in confirmed cases reported by the [Oxford COVID-19 Government Response Tracker (OxCGRT)](https://www.bsg.ox.ac.uk/research/research-projects/coronavirus-government-response-tracker).

The latest data, including the latest confirmed cases ('ConfirmedCases') can be find here: https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv

In [ ]:
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
df = pd.read_csv(DATA_URL,
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 error_bad_lines=False)

In [ ]:
df.sample(3)

### Daily change in confirmed cases
The daily change in confirmed cases can be computed like this:

In [ ]:
df["DailyChangeConfirmedCases"] = df.groupby(["CountryName", "RegionName"]).ConfirmedCases.diff().fillna(0)

For instance, for country **United States**, region **California**, the latest available changes in confirmed cases are:

In [ ]:
california_df = df[(df.CountryName == "United States") & (df.RegionName == "California")]

In [ ]:
california_df[["CountryName", "RegionName", "Date", "ConfirmedCases", "DailyChangeConfirmedCases"]].tail(5)

# Training a model

## Copy the data locally

In [ ]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
DATA_FILE = 'data/OxCGRT_latest.csv'

# Download the data set
data = requests.get(DATA_URL)

# Persist the data set locally in order to use it after submission to make predictions,
# as the sandbox won't have access to the internet anymore.
open(DATA_FILE, 'wb').write(data.content)

## Train

In [ ]:
# Consider the data after this date is not known yet
CUTOFF_DATE = pd.to_datetime("2020-07-31", format='%Y-%m-%d')

In [ ]:
# Reload the module to get the latest changes
import xprize_predictor
from importlib import reload
reload(xprize_predictor)
from xprize_predictor import XPrizePredictor

In [ ]:
predictor = XPrizePredictor(None, DATA_FILE, CUTOFF_DATE)

In [ ]:
%%time
predictor_model = predictor.train()

In [ ]:
predictor_model.save("trained_model.h5")

# Predicting using a model

## Load candidate model

In [ ]:
model_file = "trained_model.h5"

In [ ]:
predictor = XPrizePredictor(model_file, DATA_FILE, CUTOFF_DATE)

## Make prediction

In [ ]:
NPIS_INPUT_FILE = "../../validation/data/2020-08-01_2020-08-31_ip.csv"
prediction_input_df = pd.read_csv(EXAMPLE_INPUT_FILE,
                                  parse_dates=['Date'],
                                  encoding="ISO-8859-1")

In [ ]:
start_date = "2020-08-01"
end_date = "2020-08-31"

In [ ]:
%%time
predictor.predict(start_date, end_date, NPIS_INPUT_FILE)

In [ ]:
# Check that predictions are written correctly
!head 2020-08-01_2020-08-31.csv

In [ ]:
# Check the pediction file is valid
from validation.validation import validate_submission

errors = validate_submission(start_date, end_date, "2020-08-01_2020-08-31.csv")
if errors:
    for error in errors:
        print(error)
else:
    print("All good!")

# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [ ]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../validation/data/2020-08-01_2020-08-04_ip.csv

In [ ]:
# Check the pediction file is valid
from validation.validation import validate_submission

errors = validate_submission("2020-08-01", "2020-08-04", "2020-08-01_2020-08-04.csv")
if errors:
    for error in errors:
        print(error)
else:
    print("All good!")